In [2]:
import copy
from typing import cast

from build123d import Polygon, extrude, Pos, Compound, Box
from ocp_tessellate.utils import Color
from jupyter_cadquery import show, open_viewer, set_defaults

from kfactory import LayerEnum

from gdsfactory.component import Component
from gdsfactory.technology import DerivedLayer, LayerStack, LayerViews, LogicalLayer
from gdsfactory.typings import LayerSpecs

import gdsfactory as gf

Overwriting auto display for build123d BuildPart, BuildSketch, BuildLine, ShapeList


In [3]:
cv = open_viewer("GDS")

In [4]:
INSTANCES = {}

In [30]:
import numpy as np
def to_poly(
    component: Component,
    layer_views: LayerViews | None = None,
    layer_stack: LayerStack | None = None,
    exclude_layers: LayerSpecs | None = None,
    count_only=False,
) -> Compound:
    """Return build123d Compound.

    Args:
        component: to extrude in 3D.
        layer_views: layer colors from Klayout Layer Properties file.
            Defaults to active PDK.layer_views.
        layer_stack: contains thickness and zmin for each layer.
            Defaults to active PDK.layer_stack.
        exclude_layers: list of layer index to exclude.

    """
    from gdsfactory.pdk import (
        get_active_pdk,
        get_layer,
        get_layer_stack,
        get_layer_views,
    )

    try:
        from trimesh.creation import extrude_polygon
        from trimesh.scene import Scene
    except ImportError as e:
        print("you need to `pip install trimesh`")
        raise e

    layer_views = layer_views or get_layer_views()
    layer_stack = layer_stack or get_layer_stack()

    exclude_layers = exclude_layers or ()
    exclude_layers = [get_layer(layer) for layer in exclude_layers]

    component_with_booleans = layer_stack.get_component_with_derived_layers(component)
    polygons_per_layer = component_with_booleans.get_polygons_points(
        merge=True,
    )
    has_polygons = False

    poly_assembly = {
        "label":"GDS",
        "layers":{}
    }
    for level in layer_stack.layers.values():
        layer = level.layer

        if isinstance(layer, LogicalLayer):
            assert isinstance(layer.layer, tuple | LayerEnum)
            layer_tuple = cast(tuple[int, int], tuple(layer.layer))
        elif isinstance(layer, DerivedLayer):
            assert level.derived_layer is not None
            assert isinstance(level.derived_layer.layer, tuple | LayerEnum)
            layer_tuple = cast(tuple[int, int], tuple(level.derived_layer.layer))
        else:
            raise ValueError(f"Layer {layer!r} is not a DerivedLayer or LogicalLayer")

        layer_index = int(get_layer(layer_tuple))

        if layer_index in exclude_layers:
            continue

        if layer_index not in polygons_per_layer:
            continue

        zmin = level.zmin
        height = level.thickness
        layer_view = layer_views.get_from_tuple(layer_tuple)
        poly_assembly["layers"][layer_view.name] = {
            "zmin": zmin, 
            "height":height, 
            "color": layer_view.fill_color.as_hex(),
            "polygons":[]
        }
        assert layer_view.fill_color is not None
        if zmin is not None and layer_view.visible:
            has_polygons = True
            polygons = polygons_per_layer[layer_index]
            print(layer_view.name, len(polygons))
            for polygon in polygons:
                poly_assembly["layers"][layer_view.name]["polygons"].append(np.array(polygon))
            print()

    if not has_polygons:
        raise ValueError(
            f"{component.name!r} does not have polygons defined in the "
            f"layer_stack or layer_views for the active Pdk {get_active_pdk().name!r}"
        )
    return poly_assembly

In [31]:
from ocp_tessellate.utils import numpy_to_buffer_json
c = gf.c.straight_heater_doped_rib(length=100)
polys = to_poly(c)

numpy_to_buffer_json(polys)

Waveguide 1

SLAB90 1

VIAC 128

M1 2

VIA1 330

M2 2

VIA2 480

M3 2



dict_keys(['Waveguide', 'SLAB90', 'VIAC', 'M1', 'VIA1', 'M2', 'VIA2', 'M3'])

In [4]:
import copy
from typing import cast

from build123d import Polygon, extrude, Pos, Compound, Box
from ocp_tessellate.utils import Color

from kfactory import LayerEnum

from gdsfactory.component import Component
from gdsfactory.technology import DerivedLayer, LayerStack, LayerViews, LogicalLayer
from gdsfactory.typings import LayerSpecs

INSTANCES = {}


def _get_extruded_polygon(polygon, height, zmin, color, decimals=4, optimize=True):
    if optimize and len(polygon) == 4:

        dx0 = round(polygon[3][0] - polygon[0][0], decimals)
        dx1 = round(polygon[2][0] - polygon[1][0], decimals)
        dy0 = round(polygon[1][1] - polygon[0][1], decimals)
        dy1 = round(polygon[2][1] - polygon[3][1], decimals)
        h = round(height, decimals)

        # The viewer supports instances, so render a box only once and
        # reference the instance moved to the right position

        if INSTANCES.get((dx0, dy0, h)) is None:
            INSTANCES[(dx0, dx1, h)] = Box(dx0, dy0, h)
            reference = INSTANCES[(dx0, dx1, h)]
        else:
            reference = copy.copy(INSTANCES[(dx0, dx1, h)])

        center = (polygon[0][0] + dx0 / 2, polygon[0][1] + dy0 / 2)
        obj = Pos(*center, zmin + height / 2) * reference
        obj.color = Color(color)
        return obj

    p = Polygon(*polygon, align=None)
    obj = extrude(p, amount=-height)
    obj = Pos(0, 0, zmin) * obj
    obj.color = Color(color)
    return obj


def to_b123d(
    component: Component,
    layer_views: LayerViews | None = None,
    layer_stack: LayerStack | None = None,
    exclude_layers: LayerSpecs | None = None,
    count_only=False,
) -> Compound:
    """Return build123d Compound.

    Args:
        component: to extrude in 3D.
        layer_views: layer colors from Klayout Layer Properties file.
            Defaults to active PDK.layer_views.
        layer_stack: contains thickness and zmin for each layer.
            Defaults to active PDK.layer_stack.
        exclude_layers: list of layer index to exclude.

    """
    from gdsfactory.pdk import (
        get_active_pdk,
        get_layer,
        get_layer_stack,
        get_layer_views,
    )

    try:
        from trimesh.creation import extrude_polygon
        from trimesh.scene import Scene
    except ImportError as e:
        print("you need to `pip install trimesh`")
        raise e

    layer_views = layer_views or get_layer_views()
    layer_stack = layer_stack or get_layer_stack()

    exclude_layers = exclude_layers or ()
    exclude_layers = [get_layer(layer) for layer in exclude_layers]

    component_with_booleans = layer_stack.get_component_with_derived_layers(component)
    polygons_per_layer = component_with_booleans.get_polygons_points(
        merge=True,
    )
    has_polygons = False

    sub_assemblies = []
    assembly = Compound(label="GDS")
    for level in layer_stack.layers.values():
        layer = level.layer

        if isinstance(layer, LogicalLayer):
            assert isinstance(layer.layer, tuple | LayerEnum)
            layer_tuple = cast(tuple[int, int], tuple(layer.layer))
        elif isinstance(layer, DerivedLayer):
            assert level.derived_layer is not None
            assert isinstance(level.derived_layer.layer, tuple | LayerEnum)
            layer_tuple = cast(tuple[int, int], tuple(level.derived_layer.layer))
        else:
            raise ValueError(f"Layer {layer!r} is not a DerivedLayer or LogicalLayer")

        layer_index = int(get_layer(layer_tuple))

        if layer_index in exclude_layers:
            continue

        if layer_index not in polygons_per_layer:
            continue

        zmin = level.zmin
        layer_view = layer_views.get_from_tuple(layer_tuple)
        assert layer_view.fill_color is not None
        if zmin is not None and layer_view.visible:
            has_polygons = True
            polygons = polygons_per_layer[layer_index]
            height = level.thickness
            objects = []
            sub_assembly = Compound(label=str(f"{layer_view.name} ({round(zmin,4)})"))
            print(layer_view.name, len(polygons))
            if count_only:
                sub_assembly.childern=[]
            else:
                count = 0
                for polygon in polygons:
                    count += 1
                    if count % 100 == 0:
                        print(".", end="", flush=True)
                    obj = _get_extruded_polygon(
                        polygon,
                        height,
                        zmin,
                        layer_view.fill_color.as_rgb_tuple(),
                        optimize=True,
                    )
                    objects.append(obj)
                sub_assembly.children = objects
                print()
        sub_assemblies.append(sub_assembly)
    assembly.children = sub_assemblies

    if not has_polygons:
        raise ValueError(
            f"{component.name!r} does not have polygons defined in the "
            f"layer_stack or layer_views for the active Pdk {get_active_pdk().name!r}"
        )
    return assembly


In [26]:
c = gf.c.straight_heater_doped_rib(length=100)
compound = to_b123d(c, count_only=False)
show(compound, progress="")

ValueError: 'straight_heater_doped_r_62b1d634' does not have polygons defined in the layer_stack or layer_views for the active Pdk 'sky130'

In [19]:
c = gf.components.straight_heater_doped_rib(
    length=320,
    nsections=3,
    cross_section="strip_rib_tip",
    cross_section_heater="rib_heater_doped",
    via_stack="via_stack_slab_npp_m3",
    via_stack_metal="via_stack_m1_mtop",
    via_stack_metal_size=(10, 10),
    via_stack_size=(10, 10),
    taper="taper_cross_section",
    heater_width=2,
    heater_gap=0.8,
    via_stack_gap=0,
    width=0.5,
    xoffset_tip1=0.2,
    xoffset_tip2=0.4,
).copy()
compound = to_b123d(c)
show(compound, progress="")

Waveguide 1
SLAB90 1
VIAC 128
M1 2
VIA1 990
M2 2
VIA2 1360
M3 2



In [21]:
c = gf.components.straight_heater_metal(length=90)
compound = to_b123d(c)
show(compound, progress="")

Waveguide 1
MH 1
VIA1 32
M2 2
VIA2 50
M3 2



In [22]:
c = gf.components.straight_heater_meander(
    length=300,
    spacing=2,
    cross_section="strip",
    heater_width=2.5,
    extension_length=15,
    layer_heater="HEATER",
    via_stack="via_stack_heater_mtop",
    heater_taper_length=10,
    taper_length=10,
    n=3,
).copy()
compound = to_b123d(c)
show(compound, progress="")

Waveguide 1
MH 1
VIA1 32
M2 2
VIA2 50
M3 2



In [23]:
bend180 = gf.components.bend_circular180()
wg_pin = gf.components.straight_pin(length=40)
wg = gf.components.straight()

# Define a map between symbols and (component, input port, output port)
symbol_to_component = {
    "A": (bend180, "o1", "o2"),
    "B": (bend180, "o2", "o1"),
    "H": (wg_pin, "o1", "o2"),
    "-": (wg, "o1", "o2"),
}

# Each character in the sequence represents a component
s = "AB-H-H-H-H-BA"
c = gf.components.component_sequence(
    sequence=s, symbol_to_component=symbol_to_component
)
compound = to_b123d(c)
show(compound, progress="")

Waveguide 1
SLAB90 4
VIAC 288
M1 8
VIA1 192
M2 8
VIA2 288
M3 8



In [24]:
c = gf.components.coupler_full(
    coupling_length=40, dx=10, dy=4.8, gap=0.5, dw=0.1, cross_section="strip"
).copy()
compound = to_b123d(c)
show(compound, progress="")

Waveguide 2



In [26]:
c = gf.components.ge_detector_straight_si_contacts(
    length=40,
    cross_section="pn_ge_detector_si_contacts",
    via_stack="via_stack_slab_m3",
    via_stack_width=10,
    via_stack_spacing=5,
    via_stack_offset=0,
    taper_length=20,
    taper_width=0.8,
    taper_cros_section="strip",
).copy()
compound = to_b123d(c)
show(compound, progress="")

Waveguide 1
SLAB90 2
GE 1
VIAC 152
M1 2
VIA1 108
M2 2
VIA2 152
M3 2



In [27]:
# BROKEN
c = gf.components.awg(
    arms=10, outputs=3, fpr_spacing=50, arm_spacing=1, cross_section="strip"
).copy()
compound = to_b123d(c)
show(compound, progress="")

Waveguide 1



In [31]:
c = gf.components.dbr(
    w1=0.45,
    w2=0.55,
    l1=0.159,
    l2=0.159,
    n=10,
    cross_section="strip",
    straight_length=0.01,
).copy()
c.draw_ports()
compound = to_b123d(c)
show(compound, progress="")

Waveguide 1



In [32]:
c = gf.components.dbr_tapered(
    length=10,
    period=0.85,
    dc=0.5,
    w1=0.4,
    w2=1,
    taper_length=20,
    fins=False,
    fin_size=(0.2, 0.05),
    cross_section="strip",
).copy()
c.draw_ports()
compound = to_b123d(c)
show(compound, progress="")

Waveguide 1



In [8]:
from sky130 import LAYER_STACK as layer_stack_sky130

In [9]:
c = gf.read.import_gds("example_sky130.gds")

In [10]:
s = to_b123d(c, layer_stack=layer_stack_sky130, count_only=False)

polydrawing_m 3626
....................................
nwelldrawing_m 22

nsdmdrawing_m 202
..
hvtpdrawing_m 22

licon1drawing_m 25348
.............................................................................................................................................................................................................................................................
li1drawing_m 2686
..........................
mcondrawing_m 13153
...................................................................................................................................
met1 1381
.............
viadrawing_m 1695
................
met2drawing_m 647
......
via2drawing_m 458
....
met3drawing_m 126
.
via3drawing_m 440
....
met4drawing_m 5

via4drawing_m 13

met5drawing_m 5



In [12]:
show(s.children[0], progress="")

In [28]:
to_b123d(c, layer_stack=layer_stack_sky130, count_only=True)

polydrawing_m 3626
nwelldrawing_m 22
nsdmdrawing_m 202
hvtpdrawing_m 22
licon1drawing_m 25348
li1drawing_m 2686
mcondrawing_m 13153
met1 1381
viadrawing_m 1695
met2drawing_m 647
via2drawing_m 458
met3drawing_m 126
via3drawing_m 440
met4drawing_m 5
via4drawing_m 13
met5drawing_m 5

